In [1]:
import mpg.mpgml.dataset.generator as mpgml_generator

2023-04-02 04:57:54.603900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
from importlib import reload
import mpg.wrapper as mpgwrapper

In [46]:
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _convert_sparse_matrix_to_sparse_tensor(X, shape_hint):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, shape_hint)


def _stack_sparse_tensors(shape_hint, *A):
    indices = []
    values = []
    for i, Z in enumerate(A):
        for L, v in zip(Z.indices, Z.values):
            indices.append([i, *L])
            values.append(v)
    return tf.sparse.SparseTensor(indices, values, shape_hint)


def _as_tensor(A, as_dense: bool, shape_hint=None):
    if as_dense:
        return tf.convert_to_tensor(A.todense())
    else:
        return _convert_sparse_matrix_to_sparse_tensor(A, shape_hint)


def _generate_instances(n, p, seed, cardinality: int, target: bool, as_graph: bool,
                        adj_matrix: bool, weight_matrix: bool, as_dense: bool):
    generator = np.random.Generator(np.random.MT19937(seed))
    graph = mpg.graph.random_graph.gnp_random_mpg(n=n, p=p, seed=seed, method="fast", loops=True,
                                                  distribution="integers", low=0, high=10, endpoint=True)
    output = None
    if as_graph:
        output = graph
    else:
        if adj_matrix and weight_matrix:
            A = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
            W = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
            if as_dense:
                output = tf.stack([A, W], axis=0)
            else:
                output = tf.cast(_stack_sparse_tensors((2, n, n), A, W), dtype=tf.float32)

        elif adj_matrix:
            output = tf.cast(_as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n)),
                             dtype=tf.float32)
        elif weight_matrix:
            output = tf.cast(
                _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n)),
                dtype=tf.float32)
    starting = tf.constant([generator.integers(0, n), generator.integers(0, 1, endpoint=True)])
    if target:
        # TODO: Add target
        return (output, starting, 1)
    else:
        return (output, 1)

def cast_all(dtype,*args):
    return tuple(tf.cast(arg, dtype) for arg in args)


def _adj_matrix_generator(n, p):
    A = np.zeros([n, n], dtype=np.uint8)
    for k in range(n):
        A[k, :] = np.random.binomial(1, p, n)
        while A[k, :].sum() == 0:
            A[k, :] = np.random.binomial(1, p, n)
    return A

def _generate_dense_instances(n, p, seeder, cardinality: int, target: bool, weight_matrix: bool, flatten: bool,
                              weight_distribution: tfp.distributions.Distribution, weight_type):
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p, dtype=tf.bool)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = weight_distribution.sample(shape, seed=seeder())
    dtype = weight_distribution.dtype
    A = tf.numpy_function(_adj_matrix_generator, inp=[n, p], Tout=tf.uint8,stateful=True)
    if flatten:
        A = tf.reshape(A, shape=(n*n,))
    A = tf.cast(A, dtype=dtype)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,), 0, n, dtype=tf.int32, seed=seeder())
    player = turn_distribution.sample((1,), seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.concat(cast_all(dtype, A, W, vertex, player), axis=0)
        else:
            output = tf.concat(cast_all(dtype, A, vertex, player), axis=0)
        if target:
            if weight_type == tf.int32 or weight_type == tf.int64:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(
                        output.numpy().astype(np.int32).tolist()),
                    inp=[output], Tout=tf.int32)
            else:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(
                        output.numpy().astype(np.float32).tolist()),
                    inp=[output], Tout=tf.float32)
            target_value = tf.reshape(tf.ensure_shape(target_value, ()), shape=(1,))
            return (tf.cast(output, dtype=tf.float32), tf.cast(target_value, dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


class MPGGeneratedDenseDataset(tf.data.Dataset):

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                target: bool = False, weight_matrix: bool = True, flatten=False, seed=None,
                weights_distribution: tfp.distributions.Distribution = None,
                weight_type : str = "int"):
        if weight_type == "int":
            weight_type= tf.int32
        elif weight_type == "float":
            weight_type = tf.float32
        elif weight_type == "double":
            weight_type = tf.float64
        elif not isinstance(weight_type, tf.DType):
            raise ValueError("weight_type must be a string or a tf.DType")
        if weights_distribution is None:
            weights_distribution = tfp.distributions.Uniform(low=-1, high=1)
        if seed is None:
            seed = np.random.randint(0, 1 << 32)

        seeder = tfp.util.SeedStream(seed, "seeding_generator")

        shape = None
        if flatten:
            if weight_matrix:
                shape = (2 * n * n + 2,)
            else:
                shape = (n * n + 2,)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32),)
        else:
            if weight_matrix:
                shape = (2, n, n)
            else:
                shape = (n, n)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))

        generated: tf.data.Dataset
        if cardinality == tf.data.INFINITE_CARDINALITY:
            generated = tf.data.Dataset.counter(start=seed, step=1)
        else:
            generated = tf.data.Dataset.range(seed, seed + cardinality)
        return generated.map(
            lambda seed: _generate_dense_instances(n, p, seeder, cardinality, target, weight_matrix, flatten,
                                                   weights_distribution, weight_type),
            num_parallel_calls=12
        )
        #    range,
        #    args=(n,p,cardinality, target, weight_matrix,flatten),
        #    output_signature=signature
        # )

    def __init__(self, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                 target: bool = False, weight_matrix: bool = True, flatten=False, seed=None,
                 weights_distribution: tfp.distributions.Distribution = None,
                 weight_type : str = "int"):
        self.n = n
        self.p = p
        self.cardinality = cardinality
        self.target = target
        self.weight_matrix = weight_matrix
        self.flatten = flatten
        self.seed = seed
        self.weights_distribution = weights_distribution

    def _permutation(self, x, P):
        if self.flatten:
            S = tf.concat(tf.reshape(tf.tensordot(P, P, axes=None), shape=(-1,)))
            return tf.concat([tf.gather(x, S, axis=0), P[x[-2]], x[-1]])

    def permutation(self, P):
        return self.map(lambda x: self._permutation(x, P))


class MPGGeneratedDataset(tf.data.Dataset):
    def _generator(n, p, cardinality: int, target: bool, as_graph: bool,
                   adj_matrix: bool, weight_matrix: bool, as_dense: bool):
        if cardinality == tf.data.INFINITE_CARDINALITY:
            seed = 0
            while True:
                yield _generate_instances(n, p, seed, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)
                seed += 1
        else:
            for sample_idx in range(cardinality):
                yield _generate_instances(n, p, sample_idx, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY, target: bool = False, as_graph: bool = False,
                adj_matrix: bool = True, weight_matrix: bool = True, as_dense: bool = True):
        shape = None
        if as_graph:
            signature = tf.TensorSpec(shape=(), dtype=mpg.MeanPayoffGraph)
        else:
            if adj_matrix and weight_matrix:
                shape = (2, n, n)
            elif adj_matrix or weight_matrix:
                shape = (n, n)
            else:
                raise ValueError("Must specify at least one of adj_matrix or weight_matrix")
        if as_dense:
            TensorSpec = tf.TensorSpec
        else:
            TensorSpec = tf.SparseTensorSpec
        signature = (TensorSpec(shape=shape), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(n, p, cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense),
            output_signature=signature
        )

    def __init__(self, n, p):
        pass


In [55]:
dataset=MPGGeneratedDenseDataset(10,0.2,target=True,weight_matrix=True,flatten=True,weights_distribution=tfp.distributions.Uniform(-10,10),weight_type="int")
transformed=dataset.batch(64).take(1024).cache().repeat()
validation_dataset=MPGGeneratedDenseDataset(10,0.2,target=True,weight_matrix=True,flatten=True,weights_distribution=tfp.distributions.Uniform(-10,10),weight_type="float").batch(64).take(12).cache()

In [56]:
for x,y in validation_dataset.take(1):
    print(y)

tf.Tensor(
[[1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]], shape=(64, 1), dtype=float32)


2023-04-04 17:47:27.951136: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [57]:
import mpg.mpgml.layers.normalisation as mpgml_norm
import mpg.mpgml.layers.augmentation as mpgml_augm
import tensorflow.keras as keras
model=keras.Sequential([
    mpgml_norm.EdgeNormalisationLayer(edges_interval=(1,-2)),
    mpgml_augm.EdgeWeightsNoiseLayer(noise_layer=keras.layers.GaussianNoise(stddev=0.01),edges_interval=(1,-2)),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.Dense(1,"sigmoid")
])

In [58]:
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=64,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/25
64/64 [==============================] - 11s 145ms/step - loss: 0.7165 - accuracy: 0.5251 - val_loss: 0.6990 - val_accuracy: 0.4909
Epoch 2/25
64/64 [==============================] - 7s 109ms/step - loss: 0.6840 - accuracy: 0.5667 - val_loss: 0.6896 - val_accuracy: 0.5378
Epoch 3/25
64/64 [==============================] - 7s 111ms/step - loss: 0.6683 - accuracy: 0.5891 - val_loss: 0.6804 - val_accuracy: 0.5729
Epoch 4/25
64/64 [==============================] - 7s 117ms/step - loss: 0.6629 - accuracy: 0.6074 - val_loss: 0.6689 - val_accuracy: 0.6016
Epoch 5/25
64/64 [==============================] - 7s 107ms/step - loss: 0.6331 - accuracy: 0.6404 - val_loss: 0.6676 - val_accuracy: 0.5990
Epoch 6/25
64/64 [==============================] - 7s 106ms/step - loss: 0.6291 - accuracy: 0.6487 - val_loss: 0.6739 - val_accuracy: 0.6042
Epoch 7/25
64/64 [==============================] - 7s 112ms/step - loss: 0.6377 - accuracy: 0.6392 - val_loss: 0.6661 - val_accuracy: 0.5924
Epoch

In [138]:
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=256,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/25
256/256 [==============================] - 154s 571ms/step - loss: 0.6104 - accuracy: 0.6830 - val_loss: 0.5711 - val_accuracy: 0.7174
Epoch 2/25
256/256 [==============================] - 130s 510ms/step - loss: 0.5925 - accuracy: 0.6843 - val_loss: 0.5641 - val_accuracy: 0.7266
Epoch 3/25
256/256 [==============================] - 125s 488ms/step - loss: 0.5786 - accuracy: 0.6942 - val_loss: 0.5572 - val_accuracy: 0.7266
Epoch 4/25
256/256 [==============================] - 116s 454ms/step - loss: 0.5746 - accuracy: 0.6968 - val_loss: 0.5532 - val_accuracy: 0.7448
Epoch 5/25
256/256 [==============================] - 2s 6ms/step - loss: 0.5543 - accuracy: 0.7164 - val_loss: 0.5561 - val_accuracy: 0.7422
Epoch 6/25
256/256 [==============================] - 2s 6ms/step - loss: 0.5666 - accuracy: 0.7048 - val_loss: 0.5574 - val_accuracy: 0.7292
Epoch 7/25
256/256 [==============================] - 2s 6ms/step - loss: 0.5582 - accuracy: 0.7106 - val_loss: 0.5586 - val_accurac

In [130]:
x,y=next(iter(transformed))
y

2023-04-04 05:43:34.998569: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.]], dtype=float32)>

In [46]:
(x)[:,-5:]

<tf.Tensor: shape=(64, 5), dtype=float32, numpy=
array([[  0.,   0.,   0.,   1.,   1.],
       [  0.,   0.,   0.,   1.,   0.],
       [  0.,   0.,  11.,   0.,   0.],
       [  0.,   0.,   0.,   5.,   1.],
       [  0.,   0.,   0.,   7.,   1.],
       [  0.,   0.,   0.,   1.,   0.],
       [  0., -94.,   0.,   4.,   1.],
       [  0.,   0.,   0.,   3.,   0.],
       [  0.,   0.,   0.,   6.,   1.],
       [ 88.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   4.,   1.],
       [  0.,   0.,   0.,   6.,   0.],
       [  0., -66., -31.,   8.,   1.],
       [  0.,   0.,   0.,   3.,   1.],
       [  0.,   0.,  25.,   4.,   1.],
       [ 61.,   0.,   0.,   6.,   0.],
       [  0.,   0., -92.,   4.,   0.],
       [-11.,   0.,   0.,   8.,   0.],
       [  0.,   0.,   0.,   2.,   1.],
       [  0.,  63.,   0.,   5.,   1.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   4.,   0.],
       [  0.,   0.,   0.,   5.,   1.],
       [  0.,   0.,   0.,   1.,   1.],
       [  0.,  

In [111]:
graph = mpg.graph.random_graph.gnp_random_mpg(10, 0.5, seed=1, method="fast", loops=True,
                                              distribution="integers", low=0, high=10, endpoint=True)

In [123]:
A = tf.convert_to_tensor(nx.adjacency_matrix(graph, weight=None).todense())

In [181]:

coo = nx.adjacency_matrix(graph).tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
Z=tf.SparseTensor(indices, coo.data, (10,10))

TypeError: binomial() takes at least 2 positional arguments (0 given)

In [27]:
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _convert_sparse_matrix_to_sparse_tensor(X, shape_hint):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, shape_hint)


def _stack_sparse_tensors(shape_hint, *A):
    indices = []
    values = []
    for i, Z in enumerate(A):
        for L, v in zip(Z.indices, Z.values):
            indices.append([i, *L])
            values.append(v)
    return tf.sparse.SparseTensor(indices, values, shape_hint)


def _as_tensor(A, as_dense: bool, shape_hint=None):
    if as_dense:
        return tf.convert_to_tensor(A.todense())
    else:
        return _convert_sparse_matrix_to_sparse_tensor(A, shape_hint)


def _generate_instances(n, p, seed, cardinality: int, target: bool, as_graph: bool,
                        adj_matrix: bool, weight_matrix: bool, as_dense: bool):
    generator = np.random.Generator(np.random.MT19937(seed))
    graph = mpg.graph.random_graph.gnp_random_mpg(n=n, p=p, seed=seed, method="fast", loops=True,
                                                  distribution="integers", low=0, high=10, endpoint=True)
    output = None
    if as_graph:
        output = graph
    else:
        if adj_matrix and weight_matrix:
            A = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
            W = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
            if as_dense:
                output = tf.stack([A, W], axis=0)
            else:
                output = tf.cast(_stack_sparse_tensors((2, n, n), A, W), dtype=tf.float32)

        elif adj_matrix:
            output = tf.cast(_as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n)),
                             dtype=tf.float32)
        elif weight_matrix:
            output = tf.cast(
                _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n)),
                dtype=tf.float32)
    starting = tf.constant([generator.integers(0, n), generator.integers(0, 1, endpoint=True)])
    if target:
        # TODO: Add target
        return (output, starting, 1)
    else:
        return (output, 1)


def _generate_dense_instances(n, p, seed, cardinality: int, target: bool, weight_matrix: bool, flatten: bool):
    seeder=tfp.util.SeedStream(seed.numpy(),"seeding_generator")
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = tf.random.uniform(shape,-100, 101, dtype=tf.int32,seed=seeder())
    adjacency_list=[]
    for k in range(n):
        adjacency_list.append(adjacency_distribution.sample((n,),seed=seeder()))
        while tf.math.reduce_all(adjacency_list[k]==0):
            adjacency_list[k] = adjacency_distribution.sample((n,),seed=seeder())
    A= tf.concat(adjacency_list,0)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,),0, n, dtype=np.int32,seed=seeder())
    player = turn_distribution.sample((1,),seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.cast(tf.concat([A, W, vertex, player], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(tf.concat([A, W, vertex, player], axis=0), dtype=tf.float32)
        if target:
            target_value=tf.py_function(lambda output:mpgwrapper.mpgcpp.winners_tensorflow_integer_matrix_flattened_cxx(output.numpy().astype(int).tolist()),inp=[output],Tout=tf.int32)
            target_value=tf.reshape(tf.ensure_shape(target_value,()),shape=(1,))
            return (output, tf.cast(target_value,dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


class MPGGeneratedDenseDataset(tf.data.Dataset):

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                target: bool = False, weight_matrix: bool = True, flatten=False,seed=None,
                weights_distribution: tfp.distributions.Distribution = None):
        if weights_distribution is None:
            weights_distribution = tfp.distributions.Uniform(low=-100, high=100)
        if seed is None:
            seed = np.random.randint(0, 1<<32)
        shape = None
        if flatten:
            if weight_matrix:
                shape = (2 * n * n + 2,)
            else:
                shape = (n * n + 2,)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32),)
        else:
            if weight_matrix:
                shape = (2, n, n)
            else:
                shape = (n, n)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))

        generated: tf.data.Dataset
        if cardinality == tf.data.INFINITE_CARDINALITY:
            generated = tf.data.Dataset.counter(start=seed, step=1)
        else:
            generated = tf.data.Dataset.range(seed,seed+cardinality)
        return generated.map(
            lambda seed: _generate_dense_instances(n, p, seed, cardinality, target, weight_matrix, flatten),
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        #    range,
        #    args=(n,p,cardinality, target, weight_matrix,flatten),
        #    output_signature=signature
        # )

    def __init__(self, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                target: bool = False, weight_matrix: bool = True, flatten=False,seed=None,
                weights_distribution: tfp.distributions.Distribution = None):
        self.n=n
        self.p=p
        self.cardinality=cardinality
        self.target=target
        self.weight_matrix=weight_matrix
        self.flatten=flatten
        self.seed=seed
        self.weights_distribution=weights_distribution


    def _permutation(self,x,P):
        if self.flatten:
            S = tf.concat(tf.reshape(tf.tensordot(P, P, axes=None), shape=(-1,)))
            return tf.concat([tf.gather(x,S,axis=0),P[x[-2]],x[-1]])

    def permutation(self,P):
        return self.map(lambda x: self._permutation(x,P))


class MPGGeneratedDataset(tf.data.Dataset):
    def _generator(n, p, cardinality: int, target: bool, as_graph: bool,
                   adj_matrix: bool, weight_matrix: bool, as_dense: bool):
        if cardinality == tf.data.INFINITE_CARDINALITY:
            seed = 0
            while True:
                yield _generate_instances(n, p, seed, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)
                seed += 1
        else:
            for sample_idx in range(cardinality):
                yield _generate_instances(n, p, sample_idx, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY, target: bool = False, as_graph: bool = False,
                adj_matrix: bool = True, weight_matrix: bool = True, as_dense: bool = True):
        shape = None
        if as_graph:
            signature = tf.TensorSpec(shape=(), dtype=mpg.MeanPayoffGraph)
        else:
            if adj_matrix and weight_matrix:
                shape = (2, n, n)
            elif adj_matrix or weight_matrix:
                shape = (n, n)
            else:
                raise ValueError("Must specify at least one of adj_matrix or weight_matrix")
        if as_dense:
            TensorSpec = tf.TensorSpec
        else:
            TensorSpec = tf.SparseTensorSpec
        signature = (TensorSpec(shape=shape), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(n, p, cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense),
            output_signature=signature
        )

    def __init__(self, n, p):
        pass


In [198]:
zip(range(2),Z.indices)

tf.Tensor([0 0], shape=(2,), dtype=int64)
tf.Tensor([0 5], shape=(2,), dtype=int64)
tf.Tensor([0 8], shape=(2,), dtype=int64)
tf.Tensor([0 9], shape=(2,), dtype=int64)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor([1 2], shape=(2,), dtype=int64)
tf.Tensor([1 3], shape=(2,), dtype=int64)
tf.Tensor([1 5], shape=(2,), dtype=int64)
tf.Tensor([1 7], shape=(2,), dtype=int64)
tf.Tensor([1 8], shape=(2,), dtype=int64)
tf.Tensor([1 9], shape=(2,), dtype=int64)
tf.Tensor([2 0], shape=(2,), dtype=int64)
tf.Tensor([2 2], shape=(2,), dtype=int64)
tf.Tensor([2 3], shape=(2,), dtype=int64)
tf.Tensor([2 4], shape=(2,), dtype=int64)
tf.Tensor([2 7], shape=(2,), dtype=int64)
tf.Tensor([2 8], shape=(2,), dtype=int64)
tf.Tensor([3 0], shape=(2,), dtype=int64)
tf.Tensor([3 2], shape=(2,), dtype=int64)
tf.Tensor([3 3], shape=(2,), dtype=int64)
tf.Tensor([3 5], shape=(2,), dtype=int64)
tf.Tensor([3 7], shape=(2,), dtype=int64)
tf.Tensor([3 8], shape=(2,), dtype=int64)
tf.Tensor([4 1], shape=(2,), dtype

In [238]:
tf.cast(Z,tf.float64)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 5]
 [0 8]
 [0 9]
 [1 0]
 [1 2]
 [1 3]
 [1 5]
 [1 7]
 [1 8]
 [1 9]
 [2 0]
 [2 2]
 [2 3]
 [2 4]
 [2 7]
 [2 8]
 [3 0]
 [3 2]
 [3 3]
 [3 5]
 [3 7]
 [3 8]
 [4 1]
 [4 4]
 [4 5]
 [4 7]
 [5 0]
 [5 2]
 [5 3]
 [5 5]
 [5 6]
 [6 1]
 [6 2]
 [6 4]
 [6 5]
 [6 7]
 [6 8]
 [6 9]
 [7 2]
 [7 4]
 [7 6]
 [7 7]
 [7 8]
 [7 9]
 [8 1]
 [8 2]
 [8 3]
 [8 7]
 [8 8]
 [8 9]
 [9 1]
 [9 4]], shape=(53, 2), dtype=int64), values=tf.Tensor(
[ 2.  8.  1.  9.  9.  1.  5.  6.  3.  1. 10.  5.  4.  1.  1.  7.  1.  0.
  3.  4. 10. 10.  8.  0.  0.  3.  3.  6. 10.  8. 10.  1.  9.  2. 10.  5.
  0.  9.  0.  3. 10.  4.  8.  3.  8.  2.  4.  0.  9.  6.  7.  6.  1.], shape=(53,), dtype=float64), dense_shape=tf.Tensor([10 10], shape=(2,), dtype=int64))

In [2]:
from numba import jit
import networkx as nx
import numpy as np
from typing import Union, TypeVar, Type
import mpg.games.mpg as mpg


1.19 s ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import mpg.graph.random_graph as rg
G= rg.gnp_random_mpg(500,0.5,seed=27,distribution="integers",low=-10,high=10,endpoint=True)

In [27]:
%timeit mpg.optimal_strategy_pair(G)

20.2 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
import mpg.wrapper as wrapper
%timeit wrapper.optimal_strategy_pair(G)

664 ms ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:

def _get_winner(A,W,vertex,player):
    edges = []
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i,j] == 1:
                edges.append((i,j,W[i,j]))
    C=mpgwrapper.mpgcpp.winners_double_edges_cxx(edges)
    return C[player][vertex]

In [77]:
adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.57)
d=tfp.distributions.Uniform(-10,10)

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], dtype=int32)>

In [33]:
bytes(tf.constant([5]))

b'\x05\x00\x00\x00'

In [16]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
times=[]
for k in range(10):
    G=rg.gnp_random_mpg(n=1000,p=0.001,distribution="normal",loc=0,scale=1,loops=True)
    print(f"Processing Graph {k}:",flush=True)
    start=time.time()
    mpgwrapper.mpgcpp.winners_float_edges_cxx([(int(u),int(v),int(G.edges[u,v]["weight"])) for u,v in G.edges])
    end=time.time()
    print(f"Processing Time {end-start}:",flush=True)
    times.append(end-start)

Processing Graph 0:
Processing Time 0.2986409664154053:
Processing Graph 1:
Processing Time 0.2029719352722168:
Processing Graph 2:
Processing Time 0.2950937747955322:
Processing Graph 3:
Processing Time 0.012552738189697266:
Processing Graph 4:
Processing Time 0.24564290046691895:
Processing Graph 5:
Processing Time 0.01729297637939453:
Processing Graph 6:
Processing Time 0.0193173885345459:
Processing Graph 7:
Processing Time 0.3035721778869629:
Processing Graph 8:
Processing Time 0.07518315315246582:
Processing Graph 9:
Processing Time 0.2042698860168457:


In [1]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _adj_matrix_generator(n,p):
    
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while C.sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A

def _generate_dense_instances(n, p, seeder, cardinality: int, target: bool, weight_matrix: bool, flatten: bool,
                              weight_distribution: tfp.distributions.Distribution, weight_type):
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p,dtype=tf.bool)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = weight_distribution.sample(shape, seed=seeder())
    dtype=weight_distribution.dtype
    A=tf.numpy_function(_adj_matrix_generator,inp=[n,p],Tout=tf.uint8,stateful=False)
    if flatten:
        A=tf.reshape(A,shape=(-1,))
    A = tf.cast(A,dtype=dtype)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,), 0, n, dtype=tf.int32, seed=seeder())
    player = turn_distribution.sample((1,), seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.concat(cast_all(dtype,A, W, vertex, player), axis=0)
        else:
            output = tf.concat(cast_all(dtype,A, vertex, player), axis=0)
        if target:
            if weight_type == tf.int32 or weight_type == tf.int64:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(output.numpy().astype(np.int32).tolist()),
                    inp=[output], Tout=tf.int32)
            else:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(output.numpy().astype(np.float32).tolist()),
                    inp=[output], Tout=tf.float32)
            target_value = tf.reshape(tf.ensure_shape(target_value, ()), shape=(1,))
            return (tf.cast(output,dtype=tf.float32), tf.cast(target_value, dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


def cast_all(dtype,*args):
    return tuple(tf.cast(arg, dtype) for arg in args)
times=[]
with tf.device("/device:gpu:0") as device:
    for k in range(10):
        print(f"Processing Graph {k}:",flush=True)
        seeder=tfp.util.SeedStream(k,"k")
        start=time.time()
        _generate_dense_instances(n=200,p=1,seeder=seeder,cardinality=1,target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")
        end=time.time()
        print(f"Processing Time {end-start}:",flush=True)
        times.append(end-start)

2023-04-04 17:21:21.398851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Processing Graph 0:
Processing Time 0.1242368221282959:
Processing Graph 1:
Processing Time 0.015812397003173828:
Processing Graph 2:
Processing Time 0.016327857971191406:
Processing Graph 3:
Processing Time 0.01727604866027832:
Processing Graph 4:
Processing Time 0.016505002975463867:
Processing Graph 5:
Processing Time 0.01561427116394043:
Processing Graph 6:
Processing Time 0.018156051635742188:
Processing Graph 7:
Processing Time 0.01877140998840332:
Processing Graph 8:
Processing Time 0.01688218116760254:
Processing Graph 9:
Processing Time 0.013843297958374023:


In [20]:

def _adj_matrix_generator(n,p):
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while A[k,:].sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A
_adj_matrix_generator(100,0.5)

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 1]], dtype=uint8)

In [ ]:
n=100
X=_generate_dense_instances(n=n,p=1,seeder=seeder,cardinality=1,
                          target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")
L=[]
Z=X.numpy()
for i in range(n):
    for j in range(n):
        if Z[i+n*j] == 1:
            L.append((i,j,int(Z[i+n*j+n*n])))
a=time.time()
mpgwrapper.mpgcpp.optimal_strategy_pair_edges_cxx(L)
b=time.time()
print(b-a)

In [20]:
L=[]
Z=X.numpy()
for i in range(n):
    for j in range(n):
        if Z[i+n*j] == 1:
            L.append((i,j,int(Z[i+n*j+n*n])))
print(5)

5


In [49]:
n=100
X=_generate_dense_instances(n=n,p=0.01,seeder=seeder,cardinality=1,
                          target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")

In [50]:
X

<tf.Tensor: shape=(20002,), dtype=float32, numpy=array([ 0.,  0.,  0., ..., -0., 84.,  0.], dtype=float32)>

In [51]:
Z=X.numpy()
Z=Z[n*n:2*n*n].reshape((n,n))
Z=Z.astype(np.int32)
Z
A=X.numpy()[:n*n].reshape((n,n))

In [52]:
import networkx as nx
import mpg.games.mpg as mpg
G=nx.from_numpy_array(A,create_using=mpg.MeanPayoffGraph)
for e in G.edges:
    u,v=e
    G.edges[u,v]["weight"]=Z[u,v]

In [53]:
mpgwrapper.mean_payoffs(G)

({0: 2.0,
  1: 2.0,
  2: 2.0,
  3: -27.0,
  4: 2.0,
  5: 2.0,
  6: -27.0,
  7: -6.5,
  8: 1.0,
  9: 2.0,
  10: 2.0,
  11: 2.0,
  12: 1.0,
  13: 2.0,
  14: 1.0,
  15: 2.0,
  16: -27.0,
  17: 2.0,
  18: 2.0,
  19: 2.0,
  20: 2.0,
  21: 2.0,
  22: 2.0,
  23: 1.0,
  24: 2.0,
  25: -27.0,
  26: 2.0,
  27: 2.0,
  28: 2.0,
  29: 2.0,
  30: 1.0,
  31: 1.0,
  32: 2.0,
  33: 2.0,
  34: 2.0,
  35: -27.0,
  36: 2.0,
  37: 2.0,
  38: 2.0,
  39: 2.0,
  40: -27.0,
  41: -27.0,
  42: 1.0,
  43: 2.0,
  44: -27.0,
  45: 2.0,
  46: 2.0,
  47: 2.0,
  48: 2.0,
  49: 2.0,
  50: 2.0,
  51: 2.0,
  52: -6.5,
  53: 1.0,
  54: 2.0,
  55: 2.0,
  56: 2.0,
  57: 29.0,
  58: -27.0,
  59: 2.0,
  60: -27.0,
  61: 2.0,
  62: 2.0,
  63: 2.0,
  64: 2.0,
  65: -6.5,
  66: 2.0,
  67: 2.0,
  68: 2.0,
  69: 2.0,
  70: 2.0,
  71: 2.0,
  72: 2.0,
  73: -27.0,
  74: 2.0,
  75: -27.0,
  76: -27.0,
  77: 1.0,
  78: 2.0,
  79: 2.0,
  80: 2.0,
  81: 1.0,
  82: 2.0,
  83: -27.0,
  84: 2.0,
  85: 2.0,
  86: 2.0,
  87: 2.0,
  88: 1.0,

[2,
 7,
 7,
 7,
 -2,
 4,
 9,
 3,
 7,
 9,
 -4,
 -5,
 5,
 3,
 4,
 9,
 -1,
 -9,
 4,
 2,
 -6,
 2,
 -5,
 6,
 4,
 3,
 8,
 9,
 -8,
 -1,
 4,
 -3,
 7,
 6,
 9,
 6,
 8,
 5,
 1,
 8,
 -2,
 -8,
 1,
 5,
 1,
 -1,
 2,
 5,
 -4,
 -1,
 -2,
 1,
 9,
 -9,
 -9,
 -8,
 7,
 -9,
 -4,
 8,
 -1,
 -1,
 -4,
 9,
 -4,
 -8,
 -6,
 7,
 9,
 3,
 1,
 3,
 5,
 -3,
 -9,
 9,
 3,
 6,
 7,
 8,
 9,
 8,
 5,
 -2,
 -7,
 4,
 8,
 -7,
 -1,
 3,
 3,
 -9,
 -1,
 8,
 -7,
 6,
 -2,
 3,
 -5,
 -9,
 -6,
 2,
 -4,
 -1,
 -2,
 -8,
 4,
 4,
 8,
 -6,
 -1,
 -2,
 2,
 -3,
 -8,
 4,
 1,
 -5,
 -9,
 4,
 -9,
 -7,
 9,
 9,
 -6,
 2,
 -6,
 -9,
 5,
 1,
 -8,
 6,
 -6,
 -4,
 3,
 6,
 -6,
 5,
 -9,
 8,
 -1,
 -6,
 -4,
 -7,
 5,
 2,
 -4,
 -5,
 9,
 1,
 -4,
 7,
 7,
 -9,
 8,
 1,
 -6,
 7,
 4,
 3,
 3,
 2,
 9,
 -8,
 -8,
 5,
 -3,
 4,
 -9,
 5,
 -5,
 9,
 -6,
 -9,
 6,
 8,
 -1,
 2,
 2,
 -9,
 2,
 -3,
 9,
 -1,
 -6,
 1,
 -6,
 -9,
 -2,
 6,
 -1,
 5,
 7,
 6,
 -8,
 -5,
 7,
 -4,
 -6,
 8,
 -4,
 -7,
 -2,
 1,
 -3,
 -3,
 -2,
 -9,
 6,
 8,
 -9,
 -2,
 3,
 3,
 8,
 -1,
 -1,
 -4,
 -9,
 7,
 -4,
 -6,
 -2,
 -